In [20]:
!pip install sentence_transformers

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-large-zh-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\86183\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\86183\.cache\huggingface\hub\models--BAAI--bge-large-zh-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/30.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [8]:
sentence = ["我是一名华中科技大学的学生",
            "我是一名华中科技大学的研究生",
            "我是一名华中科技大学的博士生",
            "我是一名华中科技大学的教授",
            "我是一名华中科技大学的博士后"]

embeddings = model.encode(sentence)
print(embeddings.shape)

(5, 1024)


In [9]:
similarities = model.similarity(embeddings, embeddings)
print(similarities)

tensor([[1.0000, 0.8940, 0.8717, 0.7857, 0.7698],
        [0.8940, 1.0000, 0.8893, 0.8196, 0.8046],
        [0.8717, 0.8893, 1.0000, 0.8528, 0.8869],
        [0.7857, 0.8196, 0.8528, 1.0000, 0.8082],
        [0.7698, 0.8046, 0.8869, 0.8082, 1.0000]])


可以看到已经打印了五个embedding的相似矩阵。

Milvus中的SentenceTransformerEmbeddingFunction类可以直接使用sentence-transformers库中的模型，将文本转换为向量。

In [17]:
#实例化SentenceTransformerEmbeddingFunction类
from pymilvus import model
sentence_transformer_ef = model.dense.SentenceTransformerEmbeddingFunction(model_name="BAAI/bge-large-zh-v1.5", device="cpu")
#model_name指明模型名称，https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
#device指明设备，cpu或gpu，如果你有GPU，指明cuda:n

生成文档的嵌入，用encode_documents, 查询词的嵌入，用encode_queries。

In [18]:
docs = ["我是一名华中科技大学的本科生",
        "我是一名华中科技大学的研究生",
        "我是一名华中科技大学的博士生",
        "我是一名华中科技大学的教授",
        "我是一名华中科技大学的博士后"]

doc_embeddings = sentence_transformer_ef.encode_documents(docs)

In [19]:
print("Embeddings:", doc_embeddings)

Embeddings: [array([ 0.01794943, -0.00391062, -0.01233195, ...,  0.018736  ,
        0.00485343, -0.01004627], dtype=float32), array([ 0.01856543, -0.04237414, -0.01518301, ...,  0.01370529,
        0.02317346, -0.01424969], dtype=float32), array([ 0.0370553 , -0.00143078, -0.01127295, ...,  0.0066506 ,
        0.01011249, -0.01192932], dtype=float32), array([ 0.03445444,  0.00329653, -0.0220389 , ...,  0.01795297,
       -0.00550245, -0.02812228], dtype=float32), array([ 0.06066893,  0.0148532 , -0.02270859, ..., -0.0053867 ,
        0.0153412 , -0.00186932], dtype=float32)]


In [20]:
print("dim:", sentence_transformer_ef.dim)

dim: 1024


In [21]:
print("shape:",doc_embeddings[0].shape)

shape: (1024,)


In [22]:
query = ["我是哪里的学生",
         "我的学位是什么"]
query_embeddings = sentence_transformer_ef.encode_queries(query)

In [23]:
print("Embeddings:", query_embeddings)

Embeddings: [array([-0.00352036,  0.00869108,  0.01306175, ..., -0.00804216,
       -0.04357335, -0.07088969], dtype=float32), array([ 0.02859825, -0.02577852,  0.04080642, ...,  0.01069155,
        0.0172263 ,  0.0061195 ], dtype=float32)]


In [24]:
print("dim:", sentence_transformer_ef.dim)

dim: 1024


In [25]:
print("shape:",query_embeddings[0].shape)

shape: (1024,)
